In [26]:
import os
import sys
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pprint
#import cPickle
import pickle as pk
import numpy as np

model_file = "D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/image_caption_data/checkpoint_inception_v3/inception_v3_graph_def.pb"
#图像的描述
input_description_file = "D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/image_caption_data/results_20130124.token"
#图片输入的文件夹
input_img_dir = "F:/BaiduNetdiskDownload/flickr 30k/flickr30k-images/flickr30k-images"
#把上面的图片输入之后提取这三万张的图片的特征
output_folder = "D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/image_caption_data/feature_extraction_inception_v3"

batch_size = 100


if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
    
    #这个名字取的就很那啥，parse是解析的意思，看那个参数，意思就是解析那个toke_file
def parse_token_file(token_file):
    """Parses token file."""
    img_name_to_tokens = {}
    with open(token_file, 'r',encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        img_id, description = line.strip('\r\n').split('\t')
        #label中的东西又是1000092795.jpg#0	Two young guys....(以#+数字的形式来区分是图片的第几个描述)
        img_name, _ = img_id.split('#')
        img_name_to_tokens.setdefault(img_name, [])
        img_name_to_tokens[img_name].append(description)
    return img_name_to_tokens

img_name_to_tokens = parse_token_file(input_description_file)
all_img_names = img_name_to_tokens.keys()

In [29]:
print("num of all images: %d" % len(all_img_names))
#这里要在原基础上改动一下，加一个list套在外面
pprint.pprint(list(img_name_to_tokens.keys())[0:10])

#这个就是将图片描述做成一个字典，字典就是查阅功能啊，
pprint.pprint(img_name_to_tokens['2778832101.jpg'])

num of all images: 31783
['1000092795.jpg',
 '10002456.jpg',
 '1000268201.jpg',
 '1000344755.jpg',
 '1000366164.jpg',
 '1000523639.jpg',
 '1000919630.jpg',
 '10010052.jpg',
 '1001465944.jpg',
 '1001545525.jpg']
['A man in jeans is reclining on a green metal bench along a busy sidewalk and '
 'crowded street .',
 'A white male with a blue sweater and gray pants laying on a sidewalk bench .',
 'A man in a blue shirt and gray pants is sleeping on a sidewalk bench .',
 'A person is sleeping on a bench , next to cars .',
 'A man sleeping on a bench in a city area .']


In [28]:
def load_pretrained_inception_v3(model_file):
    #下面的这个是2.0版本的写法
    with tf.io.gfile.GFile(model_file, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name="")
load_pretrained_inception_v3(model_file)

In [25]:
num_batches = int(len(all_img_names) / batch_size)
if len(all_img_names) % batch_size != 0:
    num_batches += 1

with tf.Session() as sess:
    #抽取特定层的特征,这里要到迁移学习中细看
    second_to_last_tensor = sess.graph.get_tensor_by_name("pool_3:0")
    
    for i in range(num_batches):
        #就是抽取一个batch_img_names,这就是一个list啊兄弟
        batch_img_names = list(all_img_names)[i*batch_size: (i+1)*batch_size]
        
        batch_features = []
        for img_name in batch_img_names:
            #这就是做一个路径拼接，把文件夹放开头，然后后面接具体的图片的格式
            img_path = os.path.join(input_img_dir, img_name)
            #print(input_img_dir)
            print("processing img %s" % img_name)
            if not tf.io.gfile.exists(img_path):
                raise Exception("%s doesn't exists" % img_path)
                #读取图片
            img_data = tf.io.gfile.GFile(img_path, "rb").read()
            
            #输入到模型中获得某一层的特征
            feature_vector = sess.run(second_to_last_tensor,
                                      feed_dict = {"DecodeJpeg/contents:0": img_data})
            
            #把特征结果加到batch_features中去
            batch_features.append(feature_vector)
            
        #把它变成一个numpy矩阵 ，np.vstack:按垂直方向（行顺序）堆叠数组构成一个新的数组
        batch_features = np.vstack(batch_features)
        
        #"image_features-%d.pickle" % i 的意思是：image_features-0.pickle，image_features-1.pickle
        output_filename = os.path.join(output_folder, "image_features-%d.pickle" % i)
        
        print("writing to file %s" % output_filename)
        with tf.io.gfile.GFile(output_filename, 'w') as f:
            #将对象以文件的形式存放在磁盘上。dump，load 不是看名字就知道了嘛
            pk.dump((batch_img_names, batch_features), f)

processing img 1000092795.jpg
processing img 10002456.jpg
processing img 1000268201.jpg
processing img 1000344755.jpg
processing img 1000366164.jpg
processing img 1000523639.jpg
processing img 1000919630.jpg
processing img 10010052.jpg
processing img 1001465944.jpg
processing img 1001545525.jpg
processing img 1001573224.jpg
processing img 1001633352.jpg
processing img 1001773457.jpg
processing img 1001896054.jpg
processing img 100197432.jpg
processing img 100207720.jpg
processing img 1002674143.jpg
processing img 1003163366.jpg
processing img 1003420127.jpg
processing img 1003428081.jpg
processing img 100444898.jpg
processing img 1005216151.jpg
processing img 100577935.jpg
processing img 1006452823.jpg
processing img 100652400.jpg
processing img 1007129816.jpg
processing img 100716317.jpg
processing img 1007205537.jpg
processing img 1007320043.jpg
processing img 100759042.jpg
processing img 10082347.jpg
processing img 10082348.jpg
processing img 100845130.jpg
processing img 10090841.jp

processing img 109380144.jpg
processing img 1093807815.jpg
processing img 1094283901.jpg
processing img 1094462889.jpg
processing img 1094606832.jpg
processing img 1094671386.jpg
processing img 1095109931.jpg
processing img 10951809.jpg
processing img 109544260.jpg
processing img 1095476286.jpg
processing img 1095580424.jpg
processing img 1095590286.jpg
processing img 10956999.jpg
processing img 10957138.jpg
processing img 1095980313.jpg
processing img 109608573.jpg
processing img 1096097967.jpg
processing img 1096165011.jpg
processing img 1096305461.jpg
processing img 1096395242.jpg
processing img 109656696.jpg
processing img 109671650.jpg
processing img 1096800382.jpg
processing img 109738763.jpg
processing img 109738916.jpg
processing img 109741554.jpg
processing img 1097463464.jpg
processing img 109770830.jpg
processing img 1097710887.jpg
processing img 109823394.jpg
processing img 109823395.jpg
processing img 109823397.jpg
writing to file D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神

processing img 1167662968.jpg
processing img 1167669558.jpg
processing img 11678931.jpg
processing img 1167908324.jpg
processing img 116862076.jpg
processing img 1169307342.jpg
processing img 116976749.jpg
processing img 1170586898.jpg
processing img 1172514641.jpg
processing img 117430422.jpg
processing img 1174525839.jpg
processing img 1174629344.jpg
processing img 117478651.jpg
processing img 1175268298.jpg
processing img 1176580356.jpg
processing img 1177949654.jpg
processing img 1177994172.jpg
processing img 1178705300.jpg
processing img 117883760.jpg
processing img 1179086155.jpg
processing img 117919518.jpg
processing img 1179226886.jpg
processing img 117956563.jpg
processing img 11808546.jpg
processing img 1181708011.jpg
processing img 118187095.jpg
processing img 1182733666.jpg
processing img 118309463.jpg
processing img 1184340890.jpg
processing img 1184440714.jpg
processing img 1184910489.jpg
processing img 1184967930.jpg
processing img 1186128541.jpg
processing img 11863914

processing img 1265501253.jpg
processing img 126594141.jpg
processing img 1266008699.jpg
writing to file D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/image_caption_data/feature_extraction_inception_v3\image_features-7.pickle
processing img 1266116986.jpg
processing img 1266359954.jpg
processing img 12668600.jpg
processing img 126762293.jpg
processing img 1267711451.jpg
processing img 126772535.jpg
processing img 126823727.jpg
processing img 126824259.jpg
processing img 1269021329.jpg
processing img 1269022629.jpg
processing img 1269470943.jpg
processing img 1269844288.jpg
processing img 1271210445.jpg
processing img 127146749.jpg
processing img 127149546.jpg
processing img 1271579512.jpg
processing img 1271960365.jpg
processing img 127257426.jpg
processing img 1273001772.jpg
processing img 127332812.jpg
processing img 12736326.jpg
processing img 1274268203.jpg
processing img 1274416519.jpg
processing img 127449574.jpg
processing img 127450902.jpg
processing im

processing img 133267753.jpg
processing img 13327175.jpg
processing img 1332722096.jpg
processing img 1332790049.jpg
processing img 133280380.jpg
processing img 133280430.jpg
processing img 1332815795.jpg
processing img 1332823164.jpg
processing img 1332867233.jpg
processing img 133312329.jpg
processing img 1333788342.jpg
processing img 1333888922.jpg
processing img 13341510.jpg
processing img 1334403090.jpg
processing img 133456953.jpg
processing img 1334737240.jpg
processing img 1334892555.jpg
processing img 1335519498.jpg
processing img 1335617803.jpg
processing img 1335877186.jpg
processing img 1335877200.jpg
processing img 1336100240.jpg
processing img 1336391319.jpg
processing img 1336600636.jpg
processing img 133661473.jpg
processing img 1336772621.jpg
processing img 133684581.jpg
processing img 133767049.jpg
processing img 1337792872.jpg
processing img 1338067602.jpg
processing img 1338219999.jpg
processing img 1338243125.jpg
processing img 1338282180.jpg
processing img 1338523

...

processing img 2550071408.jpg
processing img 2550109269.jpg
processing img 2550254529.jpg
processing img 2550413325.jpg
processing img 2550645926.jpg
processing img 2550646414.jpg
processing img 2550669018.jpg
processing img 2550710624.jpg
processing img 2550731326.jpg
processing img 255073988.jpg
processing img 2550756380.jpg
processing img 255091927.jpg
processing img 255091930.jpg
processing img 2551007516.jpg
processing img 2551344688.jpg
processing img 2551602773.jpg
processing img 2551632823.jpg
processing img 2551697465.jpg
processing img 2552207770.jpg
processing img 2552419863.jpg
processing img 2552438538.jpg
processing img 255266148.jpg
processing img 2552723989.jpg
processing img 2552816307.jpg
processing img 2552931945.jpg
processing img 2552949275.jpg
processing img 2553024095.jpg
processing img 2553089098.jpg
processing img 2553131080.jpg
processing img 2553188198.jpg
processing img 255330891.jpg
processing img 25534384.jpg
processing img 2553550034.jpg
processing img 25

processing img 2582413611.jpg
processing img 258275599.jpg
processing img 2583001715.jpg
processing img 258309347.jpg
processing img 2583217692.jpg
processing img 2583635522.jpg
processing img 2583755519.jpg
processing img 2584020755.jpg
processing img 258434606.jpg
processing img 2584412512.jpg
processing img 2584487952.jpg
processing img 2584691004.jpg
processing img 258476074.jpg
processing img 258489513.jpg
processing img 2584957647.jpg
processing img 258501174.jpg
processing img 2585141045.jpg
processing img 2585429487.jpg
processing img 2585848069.jpg
processing img 2585970434.jpg
processing img 2586028627.jpg
processing img 2586532797.jpg
processing img 2586533475.jpg
processing img 2586695332.jpg
processing img 2586911841.jpg
processing img 2587017287.jpg
processing img 2587106431.jpg
processing img 2587202919.jpg
processing img 2587625325.jpg
processing img 2587696611.jpg
processing img 2587818583.jpg
processing img 2587846523.jpg
processing img 2588003676.jpg
processing img 2

processing img 2616991671.jpg
processing img 2617155996.jpg
processing img 2617215392.jpg
processing img 2617216530.jpg
processing img 2617219856.jpg
processing img 2617242703.jpg
processing img 2617275504.jpg
processing img 26172759.jpg
processing img 261737543.jpg
processing img 2617812188.jpg
processing img 2618322793.jpg
processing img 2618538137.jpg
processing img 261883591.jpg
processing img 2618866067.jpg
processing img 2619267133.jpg
processing img 2619454551.jpg
processing img 2619547824.jpg
processing img 2619932923.jpg
processing img 2620113705.jpg
processing img 2620483456.jpg
processing img 2620517927.jpg
processing img 2620756624.jpg
processing img 2621415349.jpg
processing img 26217061.jpg
processing img 2621771656.jpg
processing img 2622131465.jpg
processing img 2622177159.jpg
processing img 2622517932.jpg
processing img 2622581408.jpg
processing img 2622624460.jpg
processing img 2622702783.jpg
writing to file D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法

processing img 2652311904.jpg
processing img 2652382453.jpg
processing img 2652522323.jpg
processing img 2652536502.jpg
processing img 265265586.jpg
processing img 2652843146.jpg
processing img 2652974155.jpg
processing img 2653034718.jpg
processing img 2653063469.jpg
processing img 2653192579.jpg
processing img 2653199219.jpg
processing img 2653493856.jpg
processing img 2653533876.jpg
processing img 2653552905.jpg
processing img 2653617835.jpg
processing img 2653631855.jpg
processing img 2653809100.jpg
processing img 2653853997.jpg
processing img 2653938322.jpg
processing img 2653940812.jpg
processing img 2654018684.jpg
processing img 265410361.jpg
processing img 2654514044.jpg
processing img 2654751253.jpg
processing img 2654943319.jpg
processing img 2655158300.jpg
processing img 2655183854.jpg
processing img 2655196158.jpg
processing img 2655267135.jpg
processing img 265528702.jpg
processing img 265561932.jpg
processing img 2655647656.jpg
processing img 2656039837.jpg
processing img

KeyboardInterrupt: 